In [36]:
## need to adjust output directory,result directory and baseline_directory before running the code

import numpy as np
import pandas as pd
import os

pa_list=[38.30,42.03,44.76]
pe=[7.5,17.5,22.5,27.5,32.5]
b=[0,10,15,20,25]
pee=7.5
kappa=2.094215255 
zeta              = 1.66e-4*1e11

pa=pa_list[1]

output_directory="section7_2_78sites\\"
baseline_directory=f"section7_2_78sites\\gams\\p_a_{pa}_p_e_{pe[0]}"
gams_directory="section7_2_78sites\\gams\\"

In [37]:
def read_file(
    result_directory
):
    
    os.chdir(result_directory)
    
    dfz= pd.read_csv('amazon_data_z.dat', delimiter='\t')
    dfz=dfz.drop('T/R ', axis=1)
    dfz_np =dfz.to_numpy()[1:]

    dft= pd.read_csv('theta.txt',header=None)
    dft_np=dft.to_numpy().T

    dfx= pd.read_csv('amazon_data_x.dat', delimiter='\t')
    dfx=dfx.drop('T   ', axis=1)
    dfx_np =dfx.to_numpy()
    dfxdot=np.diff(dfx_np,axis=0)

    dfu= pd.read_csv('amazon_data_u.dat', delimiter='\t')
    dfu=dfu.drop('T/R ', axis=1)
    dfu_np =dfu.to_numpy()

    dfv= pd.read_csv('amazon_data_v.dat', delimiter='\t')
    dfv=dfv.drop('T/R ', axis=1)
    dfv_np =dfv.to_numpy()
    
    return(
        dfz_np,
        dft_np,
        dfxdot,
        dfu_np,
        dfv_np
        
    )

## Table 1

In [38]:
results = []
for j in range(5):

    order=j
    
    result_directory=gams_directory+f"p_a_{pa}_p_e_{pe[order]}"
    
    (   dfz_np,
        dft_np,
        dfxdot,
        dfu_np,
        dfv_np
        )   =   read_file(result_directory)
    
    
    results_AO = []
    for i in range(200):
        result_AO =pa*np.dot(dfz_np[i], dft_np[0])/((1+0.02)**(i+1))
        results_AO.append(result_AO)
    total_AO = np.sum(results_AO)
    
    results_NT = []
    for i in range(200):
        result_NT =-b[order]*(kappa*np.sum(dfz_np[i])-dfxdot[i])/((1+0.02)**(i+1))
        results_NT.append(result_NT)
    total_NT = np.sum(results_NT)
    
    results_CS = []
    for i in range(200):
        result_CS =-pee*(kappa*np.sum(dfz_np[i])-dfxdot[i])/((1+0.02)**(i+1))
        results_CS.append(result_CS)
    total_CS = np.sum(results_CS)
    
    results_AC = []
    for i in range(200):
        result_AC =(zeta/2)*(np.sum(dfu_np[i])+np.sum(dfv_np[i]))**2/((1+0.02)**(i+1))
        results_AC.append(result_AC)
    total_AC = np.sum(results_AC)
    
    total_PV=total_AO+total_NT+total_CS-total_AC
    
    iteration_results = {
        "pa":pa,
        "pe":pe[order],
        "b":b[order],
        "total_AO": total_AO,
        "total_NT": total_NT,
        "total_CS": total_CS,
        "total_AC": total_AC,
        "total_PV": total_PV
    }

    # Append the dictionary to the results list
    results.append(iteration_results)
    
results_df = pd.DataFrame(results)

# Optionally, save the DataFrame to a CSV file
results_df.to_csv(output_directory+"results_table1.csv", index=False)
latex_code = results_df.to_latex(index=False)

# Optionally, save the LaTeX code to a file
with open(output_directory+"results_table1.tex", "w") as file:
    file.write(latex_code)

C:\Users\pengyu\AppData\Local\Temp\ipykernel_13752\66244404.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_code = results_df.to_latex(index=False)


## Table 2

In [39]:


os.chdir(baseline_directory)

dfz= pd.read_csv('amazon_data_z.dat', delimiter='\t')
dfz=dfz.drop('T/R ', axis=1)
dfz_np =dfz.to_numpy()[1:]

dfx= pd.read_csv('amazon_data_x.dat', delimiter='\t')
dfx=dfx.drop('T   ', axis=1)
dfx_np =dfx.to_numpy()
dfxdot=np.diff(dfx_np,axis=0)

results_NCE_base = []
for i in range(30):
    result_NCE_base =-kappa*np.sum(dfz_np[i])+dfxdot[i]
    results_NCE_base.append(result_NCE_base)
total_NCE_base = np.sum(results_NCE_base)*100


results_table2=[]
for j in range(5):
    
    order=j
    
    result_directory=gams_directory+f"p_a_{pa}_p_e_{pe[order]}"
    
    (   dfz_np,
        dft_np,
        dfxdot,
        dfu_np,
        dfv_np
        )   =   read_file(result_directory)
    
    results_NCE = []
    for i in range(30):
        result_NCE =-kappa*np.sum(dfz_np[i])+dfxdot[i]
        results_NCE.append(result_NCE)
    total_NCE = np.sum(results_NCE)*100

    results_NT2 = []
    for i in range(30):
        result_NT2 =-b[order]*(kappa*np.sum(dfz_np[i])-dfxdot[i])
        results_NT2.append(result_NT2)
    total_NT2 = np.sum(results_NT2)

    total_EC=total_NT2/(total_NCE-total_NCE_base)*100
    
    
    iteration_results = {
        "pa":pa,
        "pe":pe[order],
        "b":b[order],
        "total_NCE": total_NCE,
        "total_NT2": total_NT2,
        "total_EC": total_EC
    }

    # Append the dictionary to the results list
    results_table2.append(iteration_results)
    
results_df2 = pd.DataFrame(results_table2)

# Optionally, save the DataFrame to a CSV file
results_df2.to_csv(output_directory+"results_table2.csv", index=False)
latex_code = results_df2.to_latex(index=False)

# Optionally, save the LaTeX code to a file
with open(output_directory+"results_table2.tex", "w") as file:
    file.write(latex_code)
    

C:\Users\pengyu\AppData\Local\Temp\ipykernel_13752\2642917604.py:45: RuntimeWarning: invalid value encountered in double_scalars
  total_EC=total_NT2/(total_NCE-total_NCE_base)*100
C:\Users\pengyu\AppData\Local\Temp\ipykernel_13752\2642917604.py:64: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_code = results_df2.to_latex(index=False)
